In [50]:
# !pip install beautifulsoup4
# !pip install requests

In [51]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

## Tier Elements

In [52]:
def scrape(filename=None):
    url = "https://little-alchemy.fandom.com/wiki/Elements_(Little_Alchemy_2)"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    result = {"elements": []}

    # remove the second table (the one with the "Special" elements)
    tables = soup.find_all("table", class_="list-table col-list icon-hover")
    tables.remove(tables[1])

    print(f"Found {len(tables)} tables.")

    tier = 0
    for table in tables:
        try:
            rows = table.find_all("tr")[1:] 

            for row in rows:
                cols = row.find_all("td")
                if len(cols) < 2:
                    continue

                # Element name from first td > a (but only direct <a>, not inside span)
                element_name = None
                for child in cols[0].children:
                    if child.name == "a":
                        element_name = child.get("title")
                        break

                if not element_name:
                    continue  # Skip if no valid element name found
                
                # Recipes from second td > ul > li > a
                recipes = []
                recipe_ul = cols[1].find("ul")
                if recipe_ul:
                    recipe_li = recipe_ul.find_all("li")
                    for li in recipe_li:
                        all_a = li.find_all("a")
                        all_a = [link.get("title") for link in all_a if link.get("title") is not None]
                        recipes.append(all_a)

                result["elements"].append({
                    "name": element_name,
                    "recipes": recipes,
                    "tier": tier
                })
                
            tier += 1
    
        except Exception as e:
            print(f"Error processing table: {e}")
            
    with open(filename, "w") as f:
        json.dump(result, f, indent=2)

    print("✅ Elements and recipes scraped successfully.")

# scrape(start=0,filename="alchemy_elements_tier.json")